In [1]:
import pandas as pd  #for converting the data into dataframes
from sklearn import datasets  #for obtaining the iris dataset
import math  #to perform math operations like log

In [2]:
#loading data from the iris dataset
iris = datasets.load_iris()

#creating a dataframe
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']
df.head()

,sl,sw,pl,pw
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:
#required result
y = iris.target
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [4]:
#Function to find label for a value
#if MIN_Value <=val < (m + Mean_Value) / 2 then it is assigned label a
#if (m + Mean_Value) <=val < Mean_Value then it is assigned label b
#if (Mean_Value) <=val < (Mean_Value + MAX_Value)/2 then it is assigned label c
#if (Mean_Value + MAX_Value)/2 <=val <= MAX_Value  then it is assigned label d

def label(val, *boundaries):
    if val < boundaries[0]:
        return 'a'
    elif val < boundaries[1]:
        return 'b'
    elif val < boundaries[2]:
        return 'c'
    else:
        return 'd'

#converting continuous data to labelled
def toLabel(df,oldFeat):
    second = df[oldFeat].mean()
    minimum = df[oldFeat].min()
    first = (minimum + second)/2
    maximum = df[oldFeat].max()
    third = (maximum + second)/2
    return df[oldFeat].apply(label, args= (first, second, third))    

In [5]:
#Convert all columns to labelled data
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')
df.head()

,sl,sw,pl,pw,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,5.1,3.5,1.4,0.2,b,c,a,a
1,4.9,3.0,1.4,0.2,a,b,a,a
2,4.7,3.2,1.3,0.2,a,c,a,a
3,4.6,3.1,1.5,0.2,a,c,a,a
4,5.0,3.6,1.4,0.2,a,c,a,a


In [6]:
#different values in feature
set(df['pw_labeled'])

{'a', 'b', 'c', 'd'}

In [7]:
#deleting redundant columns
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)

In [8]:
#adding result column to the dataframe
df['result'] = y
df.head()

,sl_labeled,sw_labeled,pl_labeled,pw_labeled,result
0,b,c,a,a,0
1,a,b,a,a,0
2,a,c,a,a,0
3,a,c,a,a,0
4,a,c,a,a,0


In [9]:
#getting the division of classes in a node
def getDivision(df):
    labels = []
    for i in range(3):
        #this will get the number of data points belonging to a certain class- (0,1,2)
        curr = len(df[df['result'] == i])
        labels.append(curr)  #appending the number of datapoints in a certain class to the label list

    return labels

## Entropy of a node

In [19]:
#finding the entropy of a given node
def entropy(df):
    #getting the division of classes in the node
    labels = getDivision(df)
    #total num of data points in the node
    totalLen = df.shape[0]
    
    #finding prob
    prob = []
    for i in range(len(labels)):
        prob.append(labels[i]/totalLen)  #probability = no of datapoints in a class / total data points
    
    #finding entropy
    Entropy = 0
    for i in range(len(labels)):
        if prob[i] != 0:
            Entropy += prob[i] * math.log2(prob[i])  #entropy = sum( prob * log2(prob) ) 
        
    return -Entropy  #entropy = - entropy as the entropy obtained above was negative

## Split Number

In [11]:
#split number
#used in calculating the gain ratio
def findSplitNo(totalLen,splitLen):
    splitNo = 0 
    #split num formula: 
    #Split Num = (-1/totalLen)(node1(len)*log(node1/total) +  node2(len)*log(node2/total))
    for i in range(len(splitLen)):
        splitNo += splitLen[i]*math.log2(splitLen[i]/totalLen)     
    splitNo = splitNo/totalLen
    
    return -splitNo #split no = -split no as the split no obtained above was negative

## Gain ratio

In [22]:
#gain ratio = infoGain/split Number
#infoGain = Entropy(old) - Entropy(split)

def findGainRatio(df,Entropy,SplitLen):
    #weighted entropy of the split
    #weighted entropy = sigma((no of elems in the class)*entropy)/total length of the data
    weightedEntropy = 0
    for i in range(len(Entropy)):
        weightedEntropy += SplitLen[i]*Entropy[i]
        
    weightedEntropy = weightedEntropy/df.shape[0]
    totalEntropy = entropy(df)
    
    infoGain = totalEntropy - weightedEntropy
    splitNo = findSplitNo(df.shape[0],SplitLen)
    if splitNo != 0:
        gainRatio = infoGain/splitNo
    else:
        gainRatio = 0
    
    return gainRatio

## Building A Tree

In [13]:
def buildTree(df,y,unusedFeatures,level):
    #level of the tree
    print("Level ",str(level))
    
    #printing the inital unused features in the tree
    print("initial features = ",unusedFeatures)
    div = getDivision(df)    
    counter=0
    numZeros = 0
    classesPresent = []
    #printing the division on the basis of classes in the node
    for x in div:
        if(x != 0):
            print("Class ",str(counter),": ",x)
            classesPresent.append(counter)
        else:
            numZeros+=1
        counter+=1
    
    #base cases
    #1 = no feature left in unusedFeatures
    if(len(unusedFeatures) == 0):
        print("LEAF NODE")
        #entropy    
        print("Entropy = ",entropy(df))
        bestClass = -1
        numInBestClass = 0
        counter = 0
        for x in div:
            if(numInBestClass < x):  #finding the best class to split upon
                bestClass = counter
                numInBestClass = x
            counter+=1
        print("predicted class = ",str(bestClass))  #predicting the best possible class
        return
    
    #2 = pure node. i.e only one class
    elif( numZeros == 2):
        print("LEAF NODE")
        print("predicted class = ",classesPresent)  #predicting the best possible class
        #ENTROPY OF PURE NODE IS 0
        print("Entropy = 0")
        return
    
    best_feature = unusedFeatures[0]
    maxGainRatio = 0
    for f in unusedFeatures:
        
        possibleVals = set(df[f])  #possible values in the fth feature
        Entropy = []
        SplitLen = []
        for i in possibleVals:
            #selecting the rows where the selected feature has a specific value
            #example i = 'a' and f = 'sl_labeled'
            #here we'll find all the rows where sl_labeled has a value 'a'
            selectedDataFrame = df.loc[df[f] == i]
            Entropy.append(entropy(selectedDataFrame))
            SplitLen.append(selectedDataFrame.shape[0])  
            
            
        #finding the gain ratio for the selected feature
        gainRatio = findGainRatio(df,Entropy,SplitLen)
        #updating the gain ratio
        if(gainRatio > maxGainRatio):
            maxGainRatio = gainRatio
            best_feature = f
                
    #printing out the best feature
    print("Selected Feature: ",best_feature)  # prints the selected best feature
    print("Entropy: ",entropy(df)) #prints the entropy of the current node
    print("Gain Ratio: ",maxGainRatio)  #prints the gain ratio of the split. Higher the gain ratio, better the split
    
    #removing the best feature from the unused_feature list
    unusedFeatures.remove(best_feature)
    print("unused features = ",unusedFeatures)  #printing all the unused features
    #recursively calling all the possible values of the feature
    vals = set(df[best_feature])
    
    #splitting the node into further nodes
    #level increases
    level+=1
    for v in vals:
        print("Splitting on ",v)
        selectedDataFrame = df.loc[df[best_feature] == v]
        print()
        print()
        print()
        print()
        buildTree(selectedDataFrame,y,unusedFeatures,level)

## Feature list

In [23]:
features = list(df.columns)
features.remove("result")
features

['sl_labeled', 'sw_labeled', 'pl_labeled', 'pw_labeled']

## Implementation of the tree

In [17]:
buildTree(df,y,features,0)

Level  0
initial features =  ['sl_labeled', 'sw_labeled', 'pl_labeled', 'pw_labeled']
Class  0 :  50
Class  1 :  50
Class  2 :  50
Selected Feature:  pw_labeled
Entropy:  1.584962500721156
Gain Ratio:  0.699638203622209
unused features =  ['sl_labeled', 'sw_labeled', 'pl_labeled']
Splitting on  d




Level  1
initial features =  ['sl_labeled', 'sw_labeled', 'pl_labeled']
Class  2 :  34
LEAF NODE
predicted class =  [2]
Entropy = 0
Splitting on  b




Level  1
initial features =  ['sl_labeled', 'sw_labeled', 'pl_labeled']
Class  1 :  10
LEAF NODE
predicted class =  [1]
Entropy = 0
Splitting on  a




Level  1
initial features =  ['sl_labeled', 'sw_labeled', 'pl_labeled']
Class  0 :  50
LEAF NODE
predicted class =  [0]
Entropy = 0
Splitting on  c




Level  1
initial features =  ['sl_labeled', 'sw_labeled', 'pl_labeled']
Class  1 :  40
Class  2 :  16
Selected Feature:  pl_labeled
Entropy:  0.863120568566631
Gain Ratio:  0.4334099495621067
unused features =  ['sl_labeled', 'sw_labeled']
Sp